In [50]:
!pip install cupy-cuda11x

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [51]:
!pip install -q -U torch numpy shapely transformers peft datasets scipy einops evaluate trl

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


^C
ERROR: Operation cancelled by user


In [52]:
!pip install accelerate@git+https://github.com/huggingface/accelerate.git@97d2168e5953fe7373a06c69c02c5a00a84d5344

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached accelerate-0.28.0.dev0-py3-none-any.whl


In [53]:
!pip install bitsandbytes

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [54]:
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    GenerationConfig,
    set_seed
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftModel
from tqdm import tqdm
from trl import SFTTrainer
import torch
import time
import pandas as pd
import numpy as np
from functools import partial
from pynvml import *
import os
from huggingface_hub import interpreter_login     
import warnings
import accelerate
import bitsandbytes as bnb

warnings.filterwarnings('ignore')

interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .


Enter your token (input will not be visible):  ·····································
Add token as git credential? (Y/n)  Y


Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.
Token has not been saved to git credential helper.
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [55]:
print(f"Accelerate version: {accelerate.__version__}")
print(f"BitsAndBytes version: {bnb.__version__}")

Accelerate version: 0.28.0.dev0
BitsAndBytes version: 0.43.1


In [56]:
os.environ['WANDB_DISABLED']="true"                 # disable Weights and Biases

def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    print(f"GPU memory occupied: {info.used//1024**2} MB.")
    
    
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"


def gen(model,p,tokenizer,maxlen=100,sample=True):
    toks = tokenizer(p, return_tensors="pt")
    res = model.generate(**toks.to("cuda"), max_new_tokens=maxlen, do_sample=sample,num_return_sequences=1,temperature=0.1,num_beams=1,top_p=0.95,).to('cpu')
    return tokenizer.batch_decode(res,skip_special_tokens=True)

## Preparing and Analysing Dataset

In [57]:
dataset = pd.read_csv('/kaggle/input/llm-design-data/df_small.csv')
dataset = dataset[['Error','Correct']]
dataset.head(2)

,Error,Correct
0,"/*\n * Copyright 2012, Homer Hsing <homer.hsi...","/*\n * Copyright 2012, Homer Hsing <homer.hsin..."
1,// Two modules are built as part of solution\n...,// Two modules are built as part of solution\n...


In [58]:
print("Total length of the dataset -",len(dataset))

for i in range(len(dataset)):
    error, correct = dataset.iloc[i,0], dataset.iloc[i,1]
    print(type(error), type(correct))
    break

Total length of the dataset - 500
<class 'str'> <class 'str'>


## Initialisation

In [59]:
%%time
seed = 42
set_seed(seed)

compute_dtype = torch.float16
bnb_config = BitsAndBytesConfig(                               # Wrapper library around CUDA custom functions - optimises 8-bit, matrix multixns
        load_in_4bit=True,                                     # using 4-bit/8-bit quantisation (here we have used 4 bit)
        bnb_4bit_quant_type='nf4',
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=False,
    )

if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'
device

CPU times: user 3.02 ms, sys: 0 ns, total: 3.02 ms
Wall time: 2.81 ms


'cuda'

In [60]:
''' LOADING THE DESIRED MODEL '''

model_name = "silverliningeda/llama-2-7b-silverliningeda-verilog-codegen"
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config)

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [61]:
''' LOAD THE CORRESPONDING TOKENIZER '''

tokenizer = AutoTokenizer.from_pretrained(model_name)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print(print_number_of_trainable_model_parameters(model),"\n")
print_gpu_utilization()

trainable model parameters: 262410240
all model parameters: 3500412928
percentage of trainable model parameters: 7.50% 

GPU memory occupied: 8722 MB.


## Baseline model

In [62]:
dash_line = '-'.join('' for x in range(100))
incorrect_code = """
    module half_adder(a,b,s,cout);

    input a,b;
    output s,cout;

    assign s  a * b;
    assign cout = a & b;

    endmodule"""

In [63]:
print("USER CASE 1")
prompt = f"module half adder"        
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
sample = model.generate(input_ids, pad_token_id=tokenizer.pad_token_id, max_length=256, temperature=0.5, top_p=0.9)
print(tokenizer.decode(sample[0]))
print(dash_line)

USER CASE 1
<s> module half adder of width 32 [
 everybody ]
  module half_adder (
    input logic [31:0] A,
    input logic [31:0] B,
    input logic cin,
    output logic [31:0] sum,
    output logic cout
  );

  logic [31:0] sum_temp;
  logic carry_temp;

  assign {carry_temp, sum_temp} = A + B + cin;
  assign sum = sum_temp;
  assign cout = carry_temp;
endmodule

module half_adder_module (
  input logic [31:0] A,
  input logic [31:0] B,
  input logic cin,
  output logic [31:0] sum,
  output logic cout
);

half_adder half_adder_inst (
  .A (A),
  .B (B),
  .cin (cin),
  .sum (sum),
  .cout (cout)
);
endmodule

endmodule
\end{code
---------------------------------------------------------------------------------------------------


In [64]:
print("USER CASE 2")
prompt = f"module full adder"        
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
sample = model.generate(input_ids, pad_token_id=tokenizer.pad_token_id, max_length=256, temperature=0.5, top_p=0.9)
print(tokenizer.decode(sample[0]))
print(dash_line)

USER CASE 2
<s> module full adder of width 32 [
 everybody ]
  module adder (
    input logic [31:0] A,
    input logic [31:0] B,
    input logic cin,
    output logic [31:0] sum,
    output logic cout
  );

  logic [31:0] sum_temp;
  logic carry_temp;

  assign {carry_temp, sum_temp} = A + B + cin;
  assign sum = sum_temp;
  assign cout = carry_temp;
endmodule

module top (
  input logic [31:0] A,
  input logic [31:0] B,
  input logic cin,
  output logic [31:0] sum,
  output logic cout
);

adder adder_inst (
  .A(A),
  .B(B),
  .cin(cin),
  .sum(sum),
  .cout(cout)
);
endmodule

endmodule
\end{code}

The `adder` module is a
---------------------------------------------------------------------------------------------------


In [65]:
print("USER CASE 3")
prompt = f"Correct the following code \n{incorrect_code}\n\nCorrect code:\n"        
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
sample = model.generate(input_ids, pad_token_id=tokenizer.pad_token_id, max_length=256, temperature=0.5, top_p=0.9)
print(tokenizer.decode(sample[0]))
print(dash_line)

USER CASE 3
<s> Correct the following code 

    module half_adder(a,b,s,cout);

    input a,b;
    output s,cout;

    assign s  a * b;
    assign cout = a & b;

    endmodule

Correct code:

module half_adder(a,b,s,cout);

input a,b;
output s,cout;

assign s = a * b;
assign cout = a & b;

endmodule

End of code

Can someone please explain why the code is corrected?

I understand that the error was in the input statement, but I don't understand why the output statements were also corrected.

Thanks in advance.

Answer:

The error in the input statement was due to the fact that the input variables `a` and `b` were not properly declared as inputs to the module. In VHDL, inputs are declared using the `input` keyword, followed by the name of the variable and its data type.

For example:

module half_adder(a: in  bit; b: in 
---------------------------------------------------------------------------------------------------


In [66]:
print("USER CASE 3")
prompt = f"Correct the following code \n{incorrect_code}\n\nCorrect code:\n"        
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
sample = model.generate(input_ids, pad_token_id=tokenizer.pad_token_id, max_length=256, temperature=0.5, top_p=0.9)
print(tokenizer.decode(sample[0]))
print(dash_line)

USER CASE 3
<s> Correct the following code 

    module half_adder(a,b,s,cout);

    input a,b;
    output s,cout;

    assign s  a * b;
    assign cout = a & b;

    endmodule

Correct code:

module half_adder(a,b,s,cout);

input a,b;
output s,cout;

assign s = a * b;
assign cout = a & b;

endmodule

End of code

Can someone please explain why the code is corrected?

I understand that the error was in the input statement, but I don't understand why the output statements were also corrected.

Thanks in advance.

Answer:

The error in the input statement was due to the fact that the input variables `a` and `b` were not properly declared as inputs to the module. In VHDL, inputs are declared using the `input` keyword, followed by the name of the variable and its data type.

For example:

module half_adder(a: in  bit; b: in 
---------------------------------------------------------------------------------------------------


## Finetuning the model

In [71]:
def create_prompt_formats(df):
    """
    Format various fields of the sample ('instruction','output')
    Then concatenate them using two newline characters 
    :param sample: Sample dictionnary
    """
    INTRO_BLURB = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
    INSTRUCTION_KEY = "### Instruct: Correct the following Verilog code"
    RESPONSE_KEY = "### Output:"
    END_KEY = "### End"
    
    blurb = f"\n{INTRO_BLURB}"
    instruction = f"{INSTRUCTION_KEY}"
    input_context = f"{df['Error']}"
    response = f"{RESPONSE_KEY}\n{df['Correct']}"
    end = f"{END_KEY}"
    
    parts = [part for part in [blurb, instruction, input_context, response, end] if part]

    formatted_prompt = "\n\n".join(parts)
    df["prompt"] = formatted_prompt

    return df


def get_max_length(model):
    conf = model.config
    max_length = None
    for length_setting in ["n_positions", "max_position_embeddings", "seq_length"]:
        max_length = getattr(model.config, length_setting, None)
        if max_length:
            print(f"Found max lenth: {max_length}")
            break
    if not max_length:
        max_length = 1024
        print(f"Using default max length: {max_length}")
    return max_length



def preprocess_batch(batch, tokenizer, max_length):
    """
    Tokenizing a batch
    """
    return tokenizer(
        batch["text"],
        max_length=max_length,
        truncation=True,
    )


def preprocess_dataset(tokenizer: AutoTokenizer, max_length: int,seed, dataset):
    """Format & tokenize it so it is ready for training
    :param tokenizer (AutoTokenizer): Model Tokenizer
    :param max_length (int): Maximum number of tokens to emit from tokenizer
    """
    
    print("Preprocessing dataset...")
    dataset = create_prompt_formats(dataset)
    return dataset
    
#     _preprocessing_function = partial(preprocess_batch, max_length=max_length, tokenizer=tokenizer)
#     dataset = dataset.map(
#         _preprocessing_function,
#         batched=True,
#     )

#     # Filter out samples that have input_ids exceeding max_length
#     dataset = dataset.filter(lambda sample: len(sample["input_ids"]) < max_length)
    
#     # Shuffle dataset
#     dataset = dataset.shuffle(seed=seed)

#     return dataset

In [72]:
dataset

,Error,Correct
0,"/*\n * Copyright 2012, Homer Hsing <homer.hsi...","/*\n * Copyright 2012, Homer Hsing <homer.hsin..."
1,// Two modules are built as part of solution\n...,// Two modules are built as part of solution\n...
2,"module half_adder(input a, b, output s0, c0);\...","module half_adder(input a, b, output s0, c0);\..."
3,"/*\n * Copyright 2012, Homer Hsing <homer.hsi...","/*\n * Copyright 2012, Homer Hsing <homer.hsin..."
4,"/*\n * Copyright 2012, Homer Hsing <homer.hsi...","/*\n * Copyright 2012, Homer Hsing <homer.hsin..."
...,...,...
495,//# 62 inputs\n //# 152 outputs\n //# 638 D-ty...,//# 62 inputs\n//# 152 outputs\n//# 638 D-type...
496,"/*\n * Copyright 2012, Homer Hsing <homer.hsi...","/*\n * Copyright 2012, Homer Hsing <homer.hsin..."
497,module ram_single #(\n parameter DATA_WIDTH=8...,module ram_single #(\n parameter DATA_WIDTH=8...
498,"/*\n * Copyright 2012, Homer Hsing <homer.hsi...","/*\n * Copyright 2012, Homer Hsing <homer.hsin..."


In [73]:
max_length = get_max_length(model)
train_dataset = create_prompt_formats(dataset)
train_dataset

Found max lenth: 4096


,Error,Correct,prompt
0,"/*\n * Copyright 2012, Homer Hsing <homer.hsi...","/*\n * Copyright 2012, Homer Hsing <homer.hsin...",\nBelow is an instruction that describes a tas...
1,// Two modules are built as part of solution\n...,// Two modules are built as part of solution\n...,\nBelow is an instruction that describes a tas...
2,"module half_adder(input a, b, output s0, c0);\...","module half_adder(input a, b, output s0, c0);\...",\nBelow is an instruction that describes a tas...
3,"/*\n * Copyright 2012, Homer Hsing <homer.hsi...","/*\n * Copyright 2012, Homer Hsing <homer.hsin...",\nBelow is an instruction that describes a tas...
4,"/*\n * Copyright 2012, Homer Hsing <homer.hsi...","/*\n * Copyright 2012, Homer Hsing <homer.hsin...",\nBelow is an instruction that describes a tas...
...,...,...,...
495,//# 62 inputs\n //# 152 outputs\n //# 638 D-ty...,//# 62 inputs\n//# 152 outputs\n//# 638 D-type...,\nBelow is an instruction that describes a tas...
496,"/*\n * Copyright 2012, Homer Hsing <homer.hsi...","/*\n * Copyright 2012, Homer Hsing <homer.hsin...",\nBelow is an instruction that describes a tas...
497,module ram_single #(\n parameter DATA_WIDTH=8...,module ram_single #(\n parameter DATA_WIDTH=8...,\nBelow is an instruction that describes a tas...
498,"/*\n * Copyright 2012, Homer Hsing <homer.hsi...","/*\n * Copyright 2012, Homer Hsing <homer.hsin...",\nBelow is an instruction that describes a tas...



Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruct: Correct the following Verilog code

0      /*\n  * Copyright 2012, Homer Hsing <homer.hsi...
1      // Two modules are built as part of solution\n...
2      module half_adder(input a, b, output s0, c0);\...
3      /*\n  * Copyright 2012, Homer Hsing <homer.hsi...
4      /*\n  * Copyright 2012, Homer Hsing <homer.hsi...
                             ...                        
495    //# 62 inputs\n //# 152 outputs\n //# 638 D-ty...
496    /*\n  * Copyright 2012, Homer Hsing <homer.hsi...
497    module ram_single #(\n  parameter DATA_WIDTH=8...
498    /*\n  * Copyright 2012, Homer Hsing <homer.hsi...
499    // This takes two signed 4 bit numbers and out...
Name: Error, Length: 500, dtype: object

### Output:
0      /*\n * Copyright 2012, Homer Hsing <homer.hsin...
1      // Two modules are built as part of solution\n...
2      module half_adder(input a, b, output s0, 